In [1]:
%pip install pyspark
%pip install spark-nlp
%pip install nltk

In [2]:
import sparknlp
from pyspark.sql import SparkSession

# Check if a SparkSession is already active
if 'spark' not in globals() or not isinstance(spark, SparkSession):
    # start the spark nlp session
    spark = sparknlp.start()
else:
    print("SparkSession is already active.")

24/09/15 12:08:55 WARN Utils: Your hostname, ouyassine-ThinkPad-L14-Gen-1 resolves to a loopback address: 127.0.1.1; using 192.168.8.145 instead (on interface wlp3s0)
24/09/15 12:08:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ouyassine/Documents/projects/data_engineering_1/venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ouyassine/.ivy2/cache
The jars for the packages stored in: /home/ouyassine/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7537160e-8344-4f15-b2f0-266639ab9b88;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.4.2 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath-java;1.12.500 in 

In [3]:
import json
import random
import pandas as pd
# reading data

with open("/home/ouyassine/Documents/projects/data_engineering_1/data/data.json") as f:
     data = json.load(f)

random.shuffle(data)

with open("/home/ouyassine/Documents/projects/data_engineering_1/data/data.json", 'w') as f:
     json.dump(data, f, indent=2)

data = pd.read_json("/home/ouyassine/Documents/projects/data_engineering_1/data/data.json")

df = spark.createDataFrame(data)
df.show(truncate=False)

+----------------------------------------------------------------------------------------------------+---------+
|title                                                                                               |topic    |
+----------------------------------------------------------------------------------------------------+---------+
|طنجة..إجهاض محاولة للتهريب غير المشروع لأدوية مصنعة من مواد مخدرة خاضعة للمراقبة الدولية            |divers   |
|سلا..توقيف شاب وسيدة بحوزتهما أقراص مهلوسة وكوكايين                                                 |divers   |
|بارالمبياد باريس..الإدريسي والنوغي تمنحان المغرب ذهبية وفضية سباق الماراثون                         |sport    |
|إدارية العاصمة تعزل رئيس بلدية القنيطرة واثنين من نوابه                                             |economie |
|النفط يهوي لأدنى مستوى في 3 سنوات                                                                   |economie |
|إعلان الرباط: وقف العدوان الإسرائيلي ينبغي أن يشكل مدخلا للتسوية النهائية والعادلة للقضية الفلس

In [4]:
%pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [5]:
from nltk.corpus import stopwords
stopwords_list = stopwords.words('arabic')

In [6]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer


# Tokenization
tokenizer = Tokenizer(inputCol="title", outputCol="tokens")

# Remove Arabic stopwords
stopwords_remover = StopWordsRemover(inputCol="tokens", outputCol="filtered_tokens", stopWords=stopwords_list)

# HashingTF to convert tokens to numerical features
hashing_tf = HashingTF(inputCol="filtered_tokens", outputCol="raw_features")

# Apply TF-IDF
idf = IDF(inputCol="raw_features", outputCol="features")

# Convert the label (topic) into numerical index
label_indexer = StringIndexer(inputCol="topic", outputCol="label")

# 4. Logistic Regression Model
lr = LogisticRegression(featuresCol="features", labelCol="label")

# 5. Build the pipeline
pipeline = Pipeline(stages=[tokenizer, stopwords_remover, hashing_tf, idf, label_indexer, lr])

In [7]:
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

In [8]:
model = pipeline.fit(train_data)

24/09/15 12:09:45 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
24/09/15 12:09:47 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
24/09/15 12:09:47 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
24/09/15 12:09:48 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
24/09/15 12:09:49 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
24/09/15 12:09:49 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
24/09/15 12:09:49 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
24/09/15 12:09:50 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
24/09/15 12:09:50 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
24/09/15 12:09:51 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
24/09/15 12:09:51 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
24/09/15 12:09:51 WARN DAGScheduler: Broadcasting larg

In [9]:
predictions = model.transform(test_data)

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print(f"Test Accuracy: {accuracy}")

24/09/15 12:10:22 WARN DAGScheduler: Broadcasting large task binary with size 4.2 MiB


Test Accuracy: 0.8520408163265306


In [10]:
model.write().overwrite().save("model")


24/09/15 12:10:28 WARN TaskSetManager: Stage 109 contains a task of very large size (4203 KiB). The maximum recommended task size is 1000 KiB.


In [11]:
predictions.select("title", "topic", "prediction").show(truncate=False)

24/09/15 12:10:35 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB


+--------------------------------------------------------------------------------------------------+---------+----------+
|title                                                                                             |topic    |prediction|
+--------------------------------------------------------------------------------------------------+---------+----------+
|38 رياضيا يمثلون المغرب في منافسات الألعاب البارالمبية “باريس 2024”                               |sport    |1.0       |
|9474 طنا هي كمية مفرغات الصيد الساحلي والتقليدي بالموانئ المغربية المتوسطية حتى متم يوليوز المنصرم|economie |2.0       |
|أحرقها صديقها..وفاة العداءة الأولمبية الأوغندية “ريبيكا تشيبتيغي”                                 |sport    |1.0       |
|أسود الأطلس يفتتحون تصفيات “الكان” بانتصار ساحق على الغابون                                       |sport    |1.0       |
|أمن “أولاد تايمة”يطلق رصاصة تحذيرية لتوقيف شخص هاجم الشرطة والمواطنين بأداة حادة                  |divers   |0.0       |
|أولمبياد غزة           